In [591]:
#Importing dependancies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import psycopg2 as ps
from psycopg2 import sql, connect
from getpass import getpass

In [592]:
#Importing Sklearn
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [593]:
#Connecting to AWS
import getpass
engine = ps.connect(
    database="postgres",
    user= input("Username: "),
    password= getpass.getpass("Password: "),
    host="highered-db.cx2ll8zuuepz.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [594]:
cr = engine.cursor()
cr.execute('SELECT * FROM highered_data_table;')
tmp = cr.fetchall()

In [595]:
#Converting to Pandas DataFrame
df = pd.DataFrame(tmp, columns= ['Year','UNITID','FACSTAT','ARANK','HRTOTLT','HRTOTLM','HRTOTLW','INSTNM','CITY','STABBR','ZIP',
                                    'CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE'])
df

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192599,2020,232618,0,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192600,2020,232618,10,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192601,2020,232618,10,6,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192602,2020,232618,40,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100


In [596]:
# #Loading the cleaned Data
# #file_path = Path('highered_merged_data.csv')
# #df = pd.read_csv(file_path)
# df.head(5)



In [597]:
# df_test = df.drop_duplicates()
df_test = df.drop(['FACSTAT', 'ARANK'], axis=1)
df_test.sample(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
4966,2020,238458,142,47,95,Carroll University,Waukesha,WI,53186,2,9,2,-88.228114,43.004527
98672,2021,200572,3,1,2,Valley City State University,Valley City,ND,58072,1,7,2,-98.003890,46.918517
49482,2019,172644,2,2,0,Wayne State University,Detroit,MI,48202,1,9,5,-83.068998,42.358994
84157,2021,187912,7,4,3,New Mexico Military Institute,Roswell,NM,88201,1,3,1,-104.524448,33.409617
51399,2019,174817,21,11,10,Saint Mary's University of Minnesota,Winona,MN,55987-1399,2,9,3,-91.695449,44.044736
45668,2021,139861,78,24,54,Georgia College & State University,Milledgeville,GA,31061,1,9,3,-83.231962,33.082787
25732,2021,102094,131,100,31,University of South Alabama,Mobile,AL,36688,1,9,4,-88.178711,30.696033
181307,2020,207403,34,18,16,Oklahoma Baptist University,Shawnee,OK,74804,2,7,2,-96.932459,35.360492
119294,2019,430935,10,6,4,Colegio de Cinematografía Artes y Television,Bayamon,PR,961,3,3,1,-66.155509,18.399507
184699,2020,215655,6,2,4,Robert Morris University,Moon Township,PA,15108,2,9,2,-80.210468,40.520415


In [598]:
#Dropping duplicates
df_test.drop_duplicates(inplace=True)

In [599]:
# df_test1 = df_test.groupby(['UNITID'], as_index=False).agg({'Year': 'first','HRTOTLT': 'sum', 'HRTOTLM': 'sum', 'HRTOTLW': 'sum'})
# df_test1.sample(10)

In [600]:
df_test1 = df_test.groupby(['UNITID', 'Year'])['HRTOTLT'].sum().unstack(fill_value=0)
#.unstack(fill_value=0)
df_test1.head(10)

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
100751,6746,6708,6928
100760,137,127,119
100812,274,256,292
100830,865,1011,1004


In [601]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 100654 to 497286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2019    3756 non-null   int64
 1   2020    3756 non-null   int64
 2   2021    3756 non-null   int64
dtypes: int64(3)
memory usage: 117.4 KB


In [602]:
df_test1.dropna()

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
...,...,...,...
496681,0,0,10
496991,0,0,7
497046,0,0,112


In [603]:
#Using drop() to delete rows based on column value
df_test1.drop(df_test1[df_test1[2019] == 0].index, inplace = True)

In [604]:
#Calculating retention rate
df_test1["Retention_Rate"] = (df_test1[2021]/df_test1[2019] * 100).round(2).astype(float)
df_test1.sample(10)

Year,2019,2020,2021,Retention_Rate
UNITID,,,,
200484,44,38,27,61.36
442806,416,633,506,121.63
175856,1276,1235,1223,95.85
454810,140,174,290,207.14
228343,338,327,342,101.18
240444,23507,25039,24669,104.94
242565,32,32,32,100.00
485272,54,101,110,203.70
233949,154,156,154,100.00


In [605]:
#Adding retention boolean
df_test1["Retention"] = np.where(df_test1['Retention_Rate'] > 99.99, True, False)
df_test1

Year,2019,2020,2021,Retention_Rate,Retention
UNITID,,,,,
100654,874,1176,1027,117.51,True
100663,9874,9961,10208,103.38,True
100690,117,100,120,102.56,True
100706,1578,1514,1534,97.21,False
100724,1026,996,970,94.54,False
...,...,...,...,...,...
494287,7,6,8,114.29,True
494436,13,13,15,115.38,True
494737,19,19,19,100.00,True


In [606]:
#Dropping Retention Rate Column
df_test1.drop(columns=["Retention"])

Year,2019,2020,2021,Retention_Rate
UNITID,,,,
100654,874,1176,1027,117.51
100663,9874,9961,10208,103.38
100690,117,100,120,102.56
100706,1578,1514,1534,97.21
100724,1026,996,970,94.54
...,...,...,...,...
494287,7,6,8,114.29
494436,13,13,15,115.38
494737,19,19,19,100.00


In [607]:
#Creating feature
X = df_test1.drop("Retention", axis=1)
X = pd.get_dummies(X)

In [608]:
#Adding a target thought? 
target = ["Retention"]

In [609]:
#Creating a target
y = df_test1.loc[:, target].copy()

In [610]:
#Decribing
X.describe()

Year,2019,2020,2021,Retention_Rate
count,3645.000000,3645.000000,3645.000000,3645.000000
mean,799.189300,776.890809,777.718519,101.006209
std,2164.002758,2098.881805,2145.416521,216.722722
min,1.000000,0.000000,0.000000,0.000000
25%,50.000000,46.000000,44.000000,86.570000
50%,212.000000,201.000000,200.000000,96.740000
75%,601.000000,584.000000,585.000000,105.740000
max,36293.000000,30347.000000,35552.000000,12900.000000


In [611]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train['Retention']))
print(Counter(y_test['Retention']))

Counter({False: 1614, True: 1119})
Counter({False: 526, True: 386})


In [612]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Counter({False: 1614, True: 1119})


In [613]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


1.0

In [614]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[526,   0],
       [  0, 386]], dtype=int64)

In [615]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      1.00      1.00      1.00      1.00      1.00       526
       True       1.00      1.00      1.00      1.00      1.00      1.00       386

avg / total       1.00      1.00      1.00      1.00      1.00      1.00       912



In [616]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Counter({False: 1614, True: 1119})


In [617]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


1.0

In [618]:
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred)
cm2

array([[526,   0],
       [  0, 386]], dtype=int64)